In [4]:
import torch
from PIL import Image
from io import BytesIO
import requests
import torchvision.transforms as transforms
import pandas as pd
from torchtext.vocab import Vocab
import collections
import ast
import torchtext

def load_img(url):
    res = requests.get(url)
#从URL加载图像,并转换为张量 
#   # 检查url是否合法
#   if not is_valid_url(url):
#     raise ValueError('Invalid URL')  
    img_data = res.content
    img = Image.open(BytesIO(img_data))

  # 转换为张量
    transform = transforms.ToTensor()
    img_tensor = transform(img)

    return img_tensor

df = pd.read_csv('7-18data.csv')
dataset=[]
for index, row in df.iterrows():
    url = row[2] 
    sales = row[4]
    url='https:'+url
    img_tensor = load_img(url)
    sample = (img_tensor,  sales)
    dataset.append(sample)
print(dataset)


[(tensor([[[0.5137, 0.5373, 0.5176,  ..., 0.7294, 0.7294, 0.7294],
         [0.6627, 0.5294, 0.5098,  ..., 0.7294, 0.7294, 0.7294],
         [0.7608, 0.6549, 0.5373,  ..., 0.7294, 0.7294, 0.7294],
         ...,
         [0.6824, 0.6863, 0.6941,  ..., 0.6510, 0.6471, 0.6431],
         [0.6824, 0.6863, 0.6941,  ..., 0.6353, 0.6314, 0.6275],
         [0.6784, 0.6824, 0.6941,  ..., 0.6235, 0.6157, 0.6118]],

        [[0.5137, 0.5373, 0.5176,  ..., 0.7333, 0.7333, 0.7333],
         [0.6627, 0.5294, 0.5098,  ..., 0.7333, 0.7333, 0.7333],
         [0.7608, 0.6549, 0.5373,  ..., 0.7333, 0.7333, 0.7333],
         ...,
         [0.6824, 0.6863, 0.6941,  ..., 0.6549, 0.6510, 0.6471],
         [0.6824, 0.6863, 0.6941,  ..., 0.6392, 0.6353, 0.6314],
         [0.6784, 0.6824, 0.6941,  ..., 0.6275, 0.6196, 0.6157]],

        [[0.4745, 0.4980, 0.4706,  ..., 0.7412, 0.7412, 0.7412],
         [0.6235, 0.4902, 0.4627,  ..., 0.7412, 0.7412, 0.7412],
         [0.7216, 0.6157, 0.4902,  ..., 0.7412, 0.7412, 

In [5]:
import torch
import torch.nn as nn 
from torchvision import models
from torchvision import models
import torch.optim as optim
import torch.utils.data as data
class SegNet(nn.Module):
    def __init__(self, num_classes=21):
        super().__init__()
        
        # Encoder (使用CNN)
        self.encoder_conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.encoder_bn1 = nn.BatchNorm2d(64)
        self.encoder_relu1 = nn.ReLU()
        
        self.encoder_conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.encoder_bn2 = nn.BatchNorm2d(128)
        self.encoder_relu2 = nn.ReLU()
        
        self.encoder_conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.encoder_bn3 = nn.BatchNorm2d(256)
        self.encoder_relu3 = nn.ReLU()

        # Decoder (调整第一层输入通道数)    
        self.decoder1 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.decoder2 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.decoder3 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.decoder4 = nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1)
        
        # Classifier
        self.classifier = nn.Conv2d(16, num_classes, kernel_size=1)

    def forward(self, x):
        # Encoder
        x = self.encoder_conv1(x)
        x = self.encoder_bn1(x)
        x = self.encoder_relu1(x)
        
        x = self.encoder_conv2(x)
        x = self.encoder_bn2(x)
        x = self.encoder_relu2(x)
        
        x = self.encoder_conv3(x)
        x = self.encoder_bn3(x)
        x = self.encoder_relu3(x)
        
        # Decoder 
        x = self.decoder1(x)
        x = self.decoder2(x)
        x = self.decoder3(x)
        x = self.decoder4(x)
        
        # Classifier
        x = self.classifier(x)
        return x
     
# 训练循环
model = SegNet(num_classes=5).train() 
dataloader = data.DataLoader(dataset)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
# for epoch in range(100):
for img, label in dataloader:
    # 设置label为正样本占比 
    label_value = 0.7  
    output = model(img)
    # 将标量label转为张量
    label_tensor = torch.tensor([label_value]) 
    label_tensor = label_tensor.expand_as(output)

    # 计算MSE Loss
    loss = criterion(output, label_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step() 
    break

# # 预测
# def preprocess(img):
# 归一化等预处理
#   return img_tensor

# img_tensor = preprocess(img)
# with torch.no_grad():
#     output = model(img_tensor) 


RuntimeError: could not create a primitive